In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../results/result.csv')

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.float_format')
pd.reset_option('display.max_colwidth')

In [19]:
df.columns

Index(['idx', 'branch_name', 'merge', 'left', 'right', 'base', 'parent test',
       'merge test', 'repo_name', 'gitmerge-ort', 'gitmerge-ort-ignorespace',
       'gitmerge-recursive-patience', 'gitmerge-recursive-minimal',
       'gitmerge-recursive-histogram', 'gitmerge-recursive-myers',
       'gitmerge-resolve', 'git-hires-merge', 'spork', 'intellimerge',
       'Equivalent gitmerge-ort gitmerge-ort-ignorespace',
       'Equivalent gitmerge-ort gitmerge-recursive-patience',
       'Equivalent gitmerge-ort gitmerge-recursive-minimal',
       'Equivalent gitmerge-ort gitmerge-recursive-histogram',
       'Equivalent gitmerge-ort gitmerge-recursive-myers',
       'Equivalent gitmerge-ort gitmerge-resolve',
       'Equivalent gitmerge-ort git-hires-merge',
       'Equivalent gitmerge-ort spork', 'Equivalent gitmerge-ort intellimerge',
       'Equivalent gitmerge-ort-ignorespace gitmerge-recursive-patience',
       'Equivalent gitmerge-ort-ignorespace gitmerge-recursive-minimal',
      

In [5]:
df.iloc[13]

idx                                                                        13
branch_name                                                 refs/heads/master
merge                                fdcc5f6727546f88afa663807134c8a02e3460fd
left                                 03e88b7fa53da1d9ce0910922bf50f8a0beccbf3
right                                14bb50651811901405b82d8f82ef8e055ba04e81
                                                       ...                   
gitmerge-recursive-myers run_time                                    0.494134
gitmerge-resolve run_time                                            4.643133
git-hires-merge run_time                                             0.340799
spork run_time                                                        4.65249
intellimerge run_time                                                3.562799
Name: 13, Length: 74, dtype: object

In [6]:
from merge_tester import MERGE_STATE
df['gitmerge-ort'].apply(lambda x: MERGE_STATE[x])[0]

<MERGE_STATE.Tests_passed: 5>

In [12]:
from validate_repos import clone_repo
repo = clone_repo(df.iloc[13]["repo_name"]) # Return a Git-Python repo object

Adobe-Consulting-Services/acs-aem-commons  : Cloning repo
Adobe-Consulting-Services/acs-aem-commons  : Finished cloning


In [13]:
# For a git to work you need to pass a name for each branch
repo.remote().fetch()
repo.git.checkout(df.iloc[13]["left"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "NAME_OF_LEFT_BRANCH", force=True)
repo.git.checkout(df.iloc[13]["right"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "NAME_OF_RIGHT_BRANCH", force=True)

''

In [ ]:
repo_path = repo.git.rev_parse("--show-toplevel")
repo_path

'/Users/benediktschesch/Git/AST-Merging-Evaluation/src/python/repos/Adobe-Consulting-Services/acs-aem-commons'

In [ ]:
import subprocess
# Returns = 1 process fails
subprocess.call(["../scripts/merge_tools/gitmerge-ort.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])

Path: /Users/benediktschesch/Git/AST-Merging-Evaluation/src/python/repos/Adobe-Consulting-Services/acs-aem-commons
Running: git merge --no-edit -s ort NAME_OF_RIGHT_BRANCH
Auto-merging bundle-twitter/pom.xml
Auto-merging bundle/pom.xml
Auto-merging bundle/src/main/java/com/adobe/acs/commons/workflow/process/impl/SyntheticWrapperWorkflowProcess.java
Auto-merging pom.xml
CONFLICT (content): Merge conflict in pom.xml
Automatic merge failed; fix conflicts and then commit the result.


Switched to branch 'NAME_OF_LEFT_BRANCH'


1

In [ ]:
subprocess.call(["cd",repo_path,"&&","git","merge","--abort"])

0

In [ ]:
# Merge Sucess (Return 0)
subprocess.call(["../scripts/merge_tools/gitmerge-ort-ignorespace.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])
# Be careful the merge has been completed the history of the left branch has now changed and you can't 
# merge again unless you delete the repo and clone it again

Path: /Users/benediktschesch/Git/AST-Merging-Evaluation/src/python/repos/Adobe-Consulting-Services/acs-aem-commons
Running: git merge --no-edit -s ort -Xignore-space-change NAME_OF_RIGHT_BRANCH
Auto-merging bundle-twitter/pom.xml
Auto-merging bundle/pom.xml
Auto-merging bundle/src/main/java/com/adobe/acs/commons/workflow/process/impl/SyntheticWrapperWorkflowProcess.java
Auto-merging pom.xml


Already on 'NAME_OF_LEFT_BRANCH'


Merge made by the 'ort' strategy.
 bundle-twitter/pom.xml                                  | 12 ++++++++++++
 bundle/pom.xml                                          | 12 ++++++++++++
 .../httpcache/engine/impl/HttpCacheEngineImpl.java      |  5 ++++-
 .../impl/ReplicateWithOptionsWorkflowProcess.java       |  3 ++-
 .../process/impl/SyntheticWrapperWorkflowProcess.java   |  3 ++-
 pom.xml                                                 | 17 ++++++++++++++++-
 6 files changed, 48 insertions(+), 4 deletions(-)
/Users/benediktschesch/Git/AST-Merging-Evaluation/src/python


0

In [32]:
df[(~df["Equivalent gitmerge-ort intellimerge"]) & df["gitmerge-ort"] != "Merge_failed"]

,idx,branch_name,merge,left,right,base,parent test,merge test,repo_name,gitmerge-ort,...,gitmerge-ort run_time,gitmerge-ort-ignorespace run_time,gitmerge-recursive-patience run_time,gitmerge-recursive-minimal run_time,gitmerge-recursive-histogram run_time,gitmerge-recursive-myers run_time,gitmerge-resolve run_time,git-hires-merge run_time,spork run_time,intellimerge run_time
0,0,refs/heads/master,c895624acfaafa2fa4f8a9ea74a61b6026f18e01,0faf7f10ae1f8f954263d24c92d2ccb484c2ba93,beb4b93e98d5eeb169dd6f6b8ea6fa8cd0acc25e,0faf7f10ae1f8f954263d24c92d2ccb484c2ba93,Tests_passed,Tests_passed,3scale/3scale_ws_api_for_java,Tests_passed,...,0.078347,0.079337,0.088289,0.086578,0.079997,0.076631,0.085703,0.028758,0.071404,2.202744
1,1,refs/heads/master,dbabe1e8bbcc49a10bc1a7c70b0e0b30ee8cd14e,c895624acfaafa2fa4f8a9ea74a61b6026f18e01,380ff2d5f51b55a4ce661d58cdd4581f9be8e9a7,c895624acfaafa2fa4f8a9ea74a61b6026f18e01,Tests_passed,Tests_passed,3scale/3scale_ws_api_for_java,Tests_passed,...,0.063635,0.045581,0.085259,0.078734,0.077879,0.125084,0.076208,0.026582,0.080857,2.063526
2,2,refs/heads/master,dfc3460abd64e6ec66da286e1f845572daa326e5,dce33107f5cd5a1b5837757a6f2005af4301ea40,ae35b5706f798733f47ce02e65291014d4513726,dce33107f5cd5a1b5837757a6f2005af4301ea40,Tests_passed,Tests_passed,3scale/3scale_ws_api_for_java,Tests_passed,...,0.081639,0.078141,0.083631,0.097037,0.084434,0.080336,0.079727,0.024618,0.188249,3.314978
3,3,refs/heads/master,bd6d850657aa0f82520b0aa958b0a30314b673d5,de13dce427aaa7401145109727dbe04e0b8c5ca9,f6b200599e977a944a91946730d6f99d288ae55d,de13dce427aaa7401145109727dbe04e0b8c5ca9,Tests_passed,Tests_passed,3scale/3scale_ws_api_for_java,Tests_passed,...,0.080487,0.074700,0.077751,0.077826,0.082005,0.080091,0.075876,0.030137,0.071771,2.037645
4,4,refs/heads/master,17e8ec087e09bc0d32c88adee32a43ae84132dbe,dfc3460abd64e6ec66da286e1f845572daa326e5,0500af924171c7364f1eaf278e83544f707b09bf,dfc3460abd64e6ec66da286e1f845572daa326e5,Tests_passed,Tests_passed,3scale/3scale_ws_api_for_java,Tests_passed,...,0.083847,0.086812,0.083623,0.079984,0.129296,0.100433,0.075980,0.023195,0.112906,3.665630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7498,7498,refs/heads/master,00e1f399ad62cb79729b6b66f8033e9dc716abc4,11f560317c06eb131e5eccacb1fc5e9a594845e1,784f1ed897967719278f0d87bb323eb204313fd9,3d5c725c61d13bdd34b641a4ee954093c6dedded,Tests_passed,Tests_passed,zsoltherpai/fluent-jdbc,Tests_passed,...,0.070012,0.074137,0.072543,0.072114,0.123546,0.072584,0.173341,0.044964,0.127322,1.894002
7499,7499,refs/heads/master,e7212c290709614b7a321947058b23dcdabed7ab,425cee073d328faa2e65f84e55b6ef963d996070,aa72825c5cb91d63a370f471baf8051362e6b885,2804daaa647fa3ea0b7757aeb7cbe0a03b65c500,Tests_passed,Tests_passed,zsoltherpai/fluent-jdbc,Tests_passed,...,0.122507,0.127685,0.131325,0.126571,0.128668,0.079697,1.026088,0.044024,4.835232,1.546544
7500,7500,refs/heads/master,c5be18ea4349ccb6e3b2a13717d25633ecbb15c0,6ba43a2fa5bbe48fea29e2117c3accf4b419dc77,e021418ae5ef767dea5e64c7badfc4f645cbc44b,de1406a68d131be78d93c389007ffeddbe224859,Tests_passed,Tests_passed,zsoltherpai/fluent-jdbc,Tests_passed,...,0.074031,0.077375,0.071025,0.073873,0.068811,0.073384,0.072817,0.039708,0.073962,1.228324
7501,7501,refs/heads/master,594e93585c4f462051b63c9e96e8e01806b68070,c268c3bdee008d359ace17ca0204901df3dc5ef8,6da6f081776a0e061db4f5af5f3901f1750b9f44,c268c3bdee008d359ace17ca0204901df3dc5ef8,Tests_passed,Tests_passed,zsoltherpai/fluent-jdbc,Tests_passed,...,0.171546,0.068578,0.070962,0.079570,0.040840,0.069760,0.036596,0.033684,0.126364,1.684654


In [33]:
# Output merges for which "Equivalent gitmerge-ort intellimerge" is true but gitmerge-ort is false
~df["Equivalent gitmerge-ort intellimerge"] & df["gitmerge-ort"] != "Merge_failed"

0       True
1       True
2       True
3       True
4       True
        ... 
7498    True
7499    True
7500    True
7501    True
7502    True
Length: 7503, dtype: bool